# 1. Importar las bibliotecas a utilizar:

In [ ]:
import cobra 
from cobra import Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
from cobra.io import read_sbml_model
import gurobipy

# 2. Definición de funciones

Defino función que calcula FVA:

In [ ]:
def fva_biomass_target(model, target):
    fvaBiomass = flux_variability_analysis(model, loopless=True)
    fvaTarget = ''
    with model:
        model.objective = target
        fvaTarget = flux_variability_analysis(model, loopless=True)
    return fvaBiomass, fvaTarget

Defino función clean_model que calcula FVA para el modelo con función objetivo la reacción de biomasa y luego con función objetivo la producción del compuesto. Las reacciones que tengan flujo mayor a cero en ambos caso se mantienen y las que no son las que deben eliminarse:

In [ ]:
def clean_model(model, fvaBiomass, fvaTarget, target):
    zeroFluxBiomass = []
    for index, row in fvaBiomass.iterrows():
        if((row['minimum'] == 0) and (row['maximum'] == 0)):
            print(index)
            zeroFluxBiomass.append(index)
            
    zeroFluxTarget = []
    for index, row in fvaTarget.iterrows():
        if((row['minimum'] == 0) and (row['maximum'] == 0) and (index != target)):
            print(index)
            zeroFluxTarget.append(index)
            
    zeroBoth = []
    for x in zeroFluxBiomass:
        if x in zeroFluxTarget: 
            zeroBoth.append(x)
    return zeroBoth   

Defino función que se encargará de eliminar reacciones del modelo:

In [ ]:
def knock_out(model, zeroBoth):
    for x in zeroBoth:
        if(x != 'ACLDC' or x!= 'ACLS' 
           or x!='BTD_RR' or x!= 'BTDt_RR' 
           or x!= 'EX_btd_RR_e'
          or x!= 'FRD2'
          or x!= 'FRD3'):
            getattr(model.reactions, x).delete()
        else: 
            print(x)

Importo los modelos creados previamente:

In [ ]:
modelCore = read_sbml_model('./models/modelCore.xml')
modelCoreA = read_sbml_model('./models/modelCoreA.xml')
modeliML = read_sbml_model('./models/modeliML.xml')
modeliMLA = read_sbml_model('./models/modeliMLA.xml')

Calculo el FVA con función objetivo biomasa y producción del compeusto:

In [ ]:
fvaBiomassCore, fvaTargetCore = fva_biomass_target(modelCore, 'EX_btd_RR_e')
fvaBiomassCoreA, fvaTargetCoreA = fva_biomass_target(modelCoreA, 'EX_btd_RR_e')
fvaBiomassiML, fvaTargetiML = fva_biomass_target(modeliML, 'EX_btd_RR_e')
fvaBiomassiMLA, fvaTargetiMLA = fva_biomass_target(modeliMLA, 'EX_btd_RR_e')

Reviso las reacciones en donde me da cero el flujo en ambos casos:

In [ ]:
ZeroBothCore = clean_model(modelCore, fvaBiomassCore, fvaTargetCore, 'EX_btd_RR_e')
ZeroBothCoreA = clean_model(modelCoreA, fvaBiomassCoreA, fvaTargetCoreA, 'EX_btd_RR_e')
ZeroBothiML = clean_model(modeliML, fvaBiomassiML, fvaTargetiML, 'EX_btd_RR_e')
ZeroBothiMLA = clean_model(modeliMLA, fvaBiomassiMLA, fvaTargetiMLA, 'EX_btd_RR_e')

Elimino las reacciones con flujo cero:

In [ ]:
knock_out(modelCore, ZeroBothCore)
knock_out(modelCoreA, ZeroBothCoreA)
knock_out(modeliML, ZeroBothiML)
knock_out(modeliMLA, ZeroBothiMLA)

Exporto los nuevos modelos reducidos en reacciones:

In [ ]:
from cobra.io import write_sbml_model
write_sbml_model(modelCore, "models/modelCore_clean.xml")
write_sbml_model(modelCoreA, "models/modelCoreA_clean.xml")
write_sbml_model(modeliML, "models/modeliML_clean.xml")
write_sbml_model(modeliMLA, "models/modeliMLA_clean.xml")